In [1]:
import pandas as pd
import numpy as np
import altair as alt
from IPython.display import HTML

# 上市公司利润分析

## 数据准备
- 从交易所取下的上市公司列表，不包括B股。
- 从东方财富取下的行业分类数据。
- 从东方财富取下的业绩快报数据。

In [22]:
def showProfile(df) :
    col_profile = pd.concat([df.dtypes, df.isna().sum()/len(df)*100, df.sample().T], axis=1 ).reset_index()
    col_profile.columns = ['列名', '类型', '缺失比%','样例']
    print(df.shape)
    return (col_profile)

In [6]:
# load data
df = pd.read_pickle('../../mydata/yjbg_y2022_profit.pkl.gz')
code = pd.read_pickle('../../mydata/ashare_basic.pkl.gz')

In [23]:
showProfile(df)

(56460, 4)


,列名,类型,缺失比%,样例
0,code,object,0.000000,300197
1,qs,object,0.000000,20121231
2,profit,float64,0.019483,215906680.37
3,year,int32,0.000000,2012


In [8]:
showProfile(code)

,列名,类型,缺失比%,样例
0,code,object,0.0,600764
1,name,object,0.0,中国海防
2,list_day,object,0.0,1996-11-04
3,market,object,0.0,主板A股
4,ex,object,0.0,上交所
5,ly,int32,0.0,1996
6,indu,object,0.0,计算机设备


- 由于公司上市会按要求披露上市前的财务数据，比如2022年上市，但业绩报告披露可能会出现前3年，甚至更久远的数据。东方财富的业绩报告显然是把这部分数据放进去了。
- 关于是否要把公司上市之前的数据去除？
  - 从年度总体可比性来说，应该去除，否则历史数据每年都会不一样。比如：2022年看2021年的数据和2023年看2021年的数据会有较大不同。所以在验证上市公司总体利润变化，是新增上市引起的还是已经上市公司利润增长引起的，就应该把以前的数据去除，不然每年的结论可能不一样。
  - 但是从分析单个公司的角度来看，不去除更合理一点。

In [59]:
# 删除上市前数据
def del_pre(df):
    code.loc[:,'ly'] = code['ly'].apply(lambda x: x if x>2010 else 2010)
    df['year'] = df['qs'].str[0:4].astype('int32')
    df = df.merge(code[['code', 'ly', 'name']])
    sel = df[df['year'] >= df['ly']]
    return sel

# 保留上市前数据
def keep_pre(df):
    code.loc[:,'ly'] = code['ly'].apply(lambda x: x if x>2010 else 2010)
    df['year'] = df['qs'].str[0:4].astype('int32')
    df = df.merge(code[['code', 'ly', 'name']])
    # sel = df[df['year'] >= df['ly']]
    return df

In [60]:
sel = del_pre(df)
showProfile(sel)

(41283, 6)


,列名,类型,缺失比%,样例
0,code,object,0.0,600853
1,qs,object,0.0,20141231
2,profit,float64,0.0,18013318.12
3,year,int32,0.0,2014
4,ly,int64,0.0,2010
5,name,object,0.0,龙建股份


In [77]:
sel = keep_pre(df)
showProfile(sel)

(56460, 6)


,列名,类型,缺失比%,样例
0,code,object,0.000000,300049
1,qs,object,0.000000,20211231
2,profit,float64,0.019483,95468878.12
3,year,int32,0.000000,2021
4,ly,int64,0.000000,2010
5,name,object,0.000000,福瑞股份


In [78]:
# 多了37%的数据。
(56460-41283)/41283

0.3676331661943173

In [62]:
# pivot the sum of each year
sel_table = sel.pivot_table(index='ly', columns='year', values='profit', aggfunc='sum')

# 分别计算出总利润，总利润增长，新上市引起的利润增长，以及已上市公司的利润增长。

# 总利润
total_sum= sel_table.sum()
# 总利润增长
total_inc = total_sum.diff()
total_inc.values[0] = 0
# 新上市引起的利润增长
li = []
for i in range(0, len(sel_table)):
    li.append(sel_table.iat[i,i])
new_inc = pd.Series(li, index=total_inc.index)
new_inc.values[0] = 0
# 已上市公司的利润增长
old_inc = total_inc - new_inc
old_inc.values[0] = 0
# 上一年基数
base = total_sum - new_inc - old_inc
base.values[0] = total_sum.values[0]

inc_df = pd.DataFrame([base, old_inc, new_inc]).T
inc_df.columns = ['base', '增长驱动', '新发驱动']

In [63]:
# 利润的一些统计值
profit_quant = sel.groupby('year')['profit'].agg(['mean', 'median', 'idxmin', 'idxmax', 'count']).reset_index()


## 上市公司利润总和的变化
- 包括所有披露数据

In [64]:
df_total = total_sum.to_frame('profit').reset_index()
alt.Chart(df_total).mark_bar().encode(
    alt.X('profit:Q').axis(format='.2s'),
    alt.Y('year:O')
).properties(
    title = '总利润变化',
    width = 'container',
    height = 340
)

alt.Chart(...)

In [65]:
# card the average increace persent
a= pow((total_sum.values[-1]/total_sum.values[0]) , (1/12))-1
avi = f'{a:.2%}'
sty = '''
  <style>
      .card_group {
        display: flex;
        flex-wrap: wrap;
        justify-content: center;
        text-align: center;
      }

      .card {
        display: flex;
        flex-direction: column;
        justify-content: center;
        align-items: center;
        background: #fff;
        border-radius: 10px;
        display: inline-block;
        height: 260px;
        margin: 1rem;
        width: 260px;
        box-shadow: 0 3px 6px rgba(0, 0, 0, 0.12), 0 3px 6px rgba(0, 0, 0, 0.24);
      }

      .card_title {
        border-bottom: 5px;
        border-bottom-style: dotted;
        border-bottom-color: grey;
        margin: 5px;
        padding: 5px;
        font-size: 2em;
      }
      .card_content {
        font-size: 4em;
        margin: 5px;
        align-self: center;
        height: 200px;

        display: flex;
        align-items: center;
        justify-content: center;
        justify-items: center;
      }
    </style>
'''

card = f'''
      {sty}
      <div class ='card_group'>
      <div class = 'card'>
      <div class = 'card_title'>总体年增长率</div>
      <div class = 'card_content'>{avi}</div>
      </div>
      </div>
'''
HTML(card)

- 上市公司总体利润保持增长,特别是疫情爆发后第2年,即2021年,出现了爆涨.
- 近12年的增长率为9%，和不保留上市前数据比，低了一个点。原因是2010年的数据增加了？这是奇怪点之一。

以下观察利润的主要统计指标:

In [79]:
alt.Chart(profit_quant).mark_bar().encode(
    alt.X('mean:Q').axis(format='.2s'),
    alt.Y('year:O')
).properties(
    title = '上市公司利润平均值变化',
    width = 'container',
    height = 340
)

alt.Chart(...)

In [67]:
alt.Chart(profit_quant).mark_bar().encode(
    alt.X('median:Q').axis(format='.2s'),
    alt.Y('year:O')
).properties(
    title = '上市公司利润中位数变化',
    width = 'container',
    height = 340
)

alt.Chart(...)

- 上市公司的平均利润总体略有增长，但2022年有较大下降，大约为10亿左右。
- 上市公司利润中位数在2022年急剧下降，略高于1亿。
- 平均值和中位数差一个数量级，说明上市公司利润的分布是左偏的。

In [68]:
mindf = sel.loc[profit_quant['idxmin'],['year','code', 'name', 'profit']].reset_index(drop = True)
base = alt.Chart(mindf).encode(
    alt.X('profit:Q').axis(format='.2s'),
    alt.Y('year:O'),
    text='name'
).properties(
    title = '历年亏损王',
    width = 'container',
    height = 340
)
base.mark_bar(color='brown') + base.mark_text(align='left', color='white')


alt.LayerChart(...)

- 上市公司亏损王呈现出不断创新高的情况。
- 谁又能想到中海远控，曾经勇巨亏王呢？
- 疫情来临之际，中海远控大翻身，而同为交通行业的大航空，却整整苦了三年。
- 看到其中的一个亏损王，感觉自己也算是亲历一段历史。

In [69]:
maxdf = sel.loc[profit_quant['idxmax'],['year','code', 'name', 'profit']].reset_index(drop = True)
base = alt.Chart(maxdf).encode(
    alt.X('profit:Q').axis(format='.2s'),
    alt.Y('year:O'),
    text='name'
).properties(
    title = '历年赢利王',
    width = 'container',
    height = 340
)
base.mark_bar() + base.mark_text(align='right', color='white')

alt.LayerChart(...)

- 赢利王没什么惊喜，宇宙第一行，永远行，疫情完全没受到影响。可怕。

## 利润构成-利润变化的驱动因素
- 上市公司上市当年的利润记为新增上市公司驱动的利润，象是给整个池子注入资本。
- 上市公司上市后第二年的利润增长做为池子本身的利润增长驱动。
- 上市公司的利润增长是不是IPO的因素更重要呢？
- 由于没有删除上市前数据，所以这个分析应该没有参考性。
- 只是用来和删除了的对比一下，这块差别较大。但似乎也没有影响基本结论。

In [70]:
alt.Chart(profit_quant).mark_bar().encode(
    alt.X('count:Q').axis(format='.0f'),
    alt.Y('year:O')
).properties(
    title = '上市公司报告数量变化',
    width = 'container',
    height = 340
)

alt.Chart(...)

- 这就是保留上市前数据最大的奇怪点：近几年的报告数居然几乎相同？！
- 最近几年IPO增长很快，利润增长是不是IPO驱动的呢？

In [71]:
inc_plotdf = inc_df.reset_index().melt(id_vars='year', var_name='cat',value_name='profit')
alt.Chart(inc_plotdf).mark_bar().encode(
    alt.X('sum(profit)').axis(format='.2s'),
    alt.Y('year:O'),
    color='cat'
).properties(
    title = '利润变化驱动因素',
    width = 'container',
    height = len(df_total)*20+80
)

alt.Chart(...)

- 可以看出，大部分时候，是已上市公司利润增长驱动的。只有少数例外。
- 其中，2015年和最近2022年，出出了已上市公司亏损，利润增长全部由当年新发公司驱动。
- 从大规律上看，上市公司的利润在爆发后，容易第二年就疲软。2017年和2021年就是个例子。
- 下图上两个驱动因素的详细对比。

In [72]:
inc_plotdf_sel = inc_plotdf[inc_plotdf['cat'] != 'base']
alt.Chart(inc_plotdf_sel).mark_bar().encode(
    alt.X('sum(profit)').axis(format='.2s'),
    alt.Y('year:O'),
    color='cat'
).properties(
    title = '两种利润增长驱动因素对比',
    width = 'container',
    height = len(df_total)*20+80
)


alt.Chart(...)

- 谁能想到疫情第2年，上市公司却出现史诗性的暴涨。
- 而到了第三年，增长不再还转为亏损。
- 这个地方也和不停留上市前数据有很大不同，2022年以前上市的公司亏损比不保留数据要大数倍，什么原因？

## 利润构成——按板块和交易所

- 这方面应该符合常识认知。

In [73]:
sel_e_m = sel.merge(code[['code', 'market', 'ex']])
sel_bymarket  =sel_e_m.groupby(['year','market'])['profit'].sum().to_frame('profit').reset_index()
alt.Chart(sel_bymarket).mark_bar().encode(
    alt.X('sum(profit)').axis(format='.2s'),
    alt.Y('year:O'),
    color='market'
).properties(
    title = '不同板块利润变化',
    width = 'container',
    height = 340
)


alt.Chart(...)

In [74]:
sel_byex  =sel_e_m.groupby(['year','ex'])['profit'].sum().to_frame('profit').reset_index()
alt.Chart(sel_byex).mark_bar().encode(
    alt.X('sum(profit)').axis(format='.2s'),
    alt.Y('year:O'),
    color='ex'
).properties(
    title = '不同交易所利润变化',
    width = 'container',
    height = 340
)


alt.Chart(...)

- 2020年对上交所打击比较大。
- 但第二年2021年，上交所又迎来狂欢。

## 利润构成-分行业看

- 一般人可能不知道，中国上市银行有多挣钱，不然无法解释为什么银行股在大部分时候都被嫌弃。
- 先看一下从2010年以来，每年的利润额前十榜上有哪些公司。

In [75]:
# 先按年份和利润额排列，然后取每年的前10名。
top10 = sel.sort_values(by=['year', 'profit'], ascending=[True, False]).groupby('year', as_index=False).nth[:10]['name'].value_counts().to_frame('count').reset_index()

alt.Chart(top10).mark_bar().encode(
    alt.X('count:Q'),
    alt.Y('name:O').sort('-x'),
).properties(
    title = '2010至2022利润额前十上榜次数统计',
    width = 'container',
    height = 440
)

alt.Chart(...)

- 每年都上榜的有6家，全是银行。而且，工商银行每年都是冠军。
- 上榜的共有19家公司，银行占了11家，如果把平安和人寿算进去，金融占了13家。
- 4家资源性公司，三桶油加一个煤炭。
- 1家交通运输企业，中远海控。应该就是2021年疫情上的榜。世事无常，它还曾夺过亏损冠军。
- 1家高科技企业，中国移动。

In [76]:
indu_counts = code.groupby('indu')['code'].count().sort_values().to_frame('counts').reset_index()
alt.Chart(indu_counts).mark_bar().encode(
    alt.Y('counts:Q').bin(step = 20),
    alt.X('count()')
).properties(
    width='container',
    height=400
)

alt.Chart(...)